In [1]:
from j2v import mag
from j2v import w2v
from j2v import predict

import numpy as np
import pandas as pd
import random
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import matplotlib.pyplot as plt
%matplotlib inline
import csv
import networkx as nx

Using Theano backend.


In [15]:
def dict_to_csv(csv_file, csv_columns, mdict):
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)#, quoting=csv.QUOTE_ALL)
        # writer.writeheader()
        for key, value in mdict.items():
            writer.writerow({csv_columns[0]: key, csv_columns[1]: value})

## PR of journals (venue citation network)

In [3]:
refs = "PaperReferences_J_C.csv"
paps = "Papers_J_C.csv"

In [4]:
vid2vname = mag.get_venue_dict()

processing /l/nx/data/haopeng/j2v/Journals.txt...
finished processing!

processing /l/nx/data/haopeng/j2v/Conferences.txt...
finished processing!



In [5]:
pid_vid_dict, _ = mag.get_pid_vid_dict(paps)

processing /l/nx/data/MicrosoftAcademicGraph/Papers_J_C.csv...
processed 1000000 lines...
processed 2000000 lines...
processed 3000000 lines...
processed 4000000 lines...
processed 5000000 lines...
processed 6000000 lines...
processed 7000000 lines...
processed 8000000 lines...
processed 9000000 lines...
processed 10000000 lines...
processed 11000000 lines...
processed 12000000 lines...
processed 13000000 lines...
processed 14000000 lines...
processed 15000000 lines...
processed 16000000 lines...
processed 17000000 lines...
processed 18000000 lines...
processed 19000000 lines...
processed 20000000 lines...
processed 21000000 lines...
processed 22000000 lines...
processed 23000000 lines...
processed 24000000 lines...
processed 25000000 lines...
processed 26000000 lines...
processed 27000000 lines...
processed 28000000 lines...
processed 29000000 lines...
processed 30000000 lines...
processed 31000000 lines...
processed 32000000 lines...
processed 33000000 lines...
processed 34000000 lin

In [6]:
vid_out_total = defaultdict(int)
vid_vid_total = defaultdict(int)
for line in mag.yield_one_line(refs, delimiter=','):
    src, des = line
    s_vid, d_vid = pid_vid_dict[src], pid_vid_dict[des]
    vid_vid_total[(s_vid, d_vid)] += 1
    vid_out_total[s_vid] += 1

processing /l/nx/data/MicrosoftAcademicGraph/PaperReferences_J_C.csv...
processed 1000000 lines...
processed 2000000 lines...
processed 3000000 lines...
processed 4000000 lines...
processed 5000000 lines...
processed 6000000 lines...
processed 7000000 lines...
processed 8000000 lines...
processed 9000000 lines...
processed 10000000 lines...
processed 11000000 lines...
processed 12000000 lines...
processed 13000000 lines...
processed 14000000 lines...
processed 15000000 lines...
processed 16000000 lines...
processed 17000000 lines...
processed 18000000 lines...
processed 19000000 lines...
processed 20000000 lines...
processed 21000000 lines...
processed 22000000 lines...
processed 23000000 lines...
processed 24000000 lines...
processed 25000000 lines...
processed 26000000 lines...
processed 27000000 lines...
processed 28000000 lines...
processed 29000000 lines...
processed 30000000 lines...
processed 31000000 lines...
processed 32000000 lines...
processed 33000000 lines...
processed 340

processed 285000000 lines...
processed 286000000 lines...
processed 287000000 lines...
processed 288000000 lines...
processed 289000000 lines...
processed 290000000 lines...
processed 291000000 lines...
processed 292000000 lines...
processed 293000000 lines...
processed 294000000 lines...
processed 295000000 lines...
processed 296000000 lines...
processed 297000000 lines...
processed 298000000 lines...
processed 299000000 lines...
processed 300000000 lines...
processed 301000000 lines...
processed 302000000 lines...
processed 303000000 lines...
processed 304000000 lines...
processed 305000000 lines...
processed 306000000 lines...
processed 307000000 lines...
processed 308000000 lines...
processed 309000000 lines...
processed 310000000 lines...
processed 311000000 lines...
processed 312000000 lines...
processed 313000000 lines...
processed 314000000 lines...
processed 315000000 lines...
processed 316000000 lines...
processed 317000000 lines...
processed 318000000 lines...
processed 3190

In [7]:
weighted_edges = []
for sid, vid in vid_vid_total:
    weight = vid_vid_total[(sid, vid)]/vid_out_total[sid]
    weighted_edges.append((sid, vid, weight))

In [8]:
g = nx.DiGraph()
g.add_weighted_edges_from(weighted_edges)
print(nx.info(g))
pr = nx.pagerank(g, alpha=0.85)

Name: 
Type: DiGraph
Number of nodes: 23304
Number of edges: 15997935
Average in degree: 686.4888
Average out degree: 686.4888


In [9]:
dict_to_csv(mag.get_path('pr_vid.csv'), ['vid', 'pr_value'], pr)

In [10]:
del weighted_edges, vid_vid_total, pid_vid_dict, vid_out_total, g

### Get top journals in each discipline

In [5]:
vid_pr = {}
for line in mag.yield_one_line('pr_vid.csv', delimiter=','):
    vid, pr = line
    vid_pr[vid] = float(pr)

processing /l/nx/data/haopeng/j2v/pr_vid.csv...
finished processing!



In [6]:
mapping = mag.load_map_jid_discipline('journal_discipline_map.csv')

14113 journals in MAG's Journal.txt were matched to UCSD data, and 29 of them are interdisciplinary journals in UCSD catelog.


In [7]:
mapping[mapping['disc_name'] == 'Interdiscipline']

,jid,jname,ucsd_jname,subd_id,subd_name,disc_id,disc_name,color
662,0655F58A,Lecture Notes in Computer Science,Lecture Notes In Computer Science,0,Interdiscipline,0,Interdiscipline,Black
1103,08260877,Environmental Toxicology and Chemistry,Environmental Toxicology And Chemistry,0,Interdiscipline,0,Interdiscipline,Black
1509,00B3075F,Environmental Monitoring and Assessment,Environmental Monitoring And Assessment,0,Interdiscipline,0,Interdiscipline,Black
1805,085C135E,Journal of Biological Chemistry,Journal Of Biological Chemistry,0,Interdiscipline,0,Interdiscipline,Black
2059,06899EEC,Journal of Theoretical Biology,Journal Of Theoretical Biology,0,Interdiscipline,0,Interdiscipline,Black
2787,00AC5F76,Journal of the Acoustical Society of America,Journal Of The Acoustical Society Of America,0,Interdiscipline,0,Interdiscipline,Black
3788,0C14A777,Industrial & Engineering Chemistry Research,Industrial And Engineering Chemistry Research,0,Interdiscipline,0,Interdiscipline,Black
3978,066C8781,Hydrobiologia,Hydrobiologia,0,Interdiscipline,0,Interdiscipline,Black
5143,0A7FA02A,Journal of Chromatography A,Journal Of Chromatography A,0,Interdiscipline,0,Interdiscipline,Black
5646,032F9796,Water Science and Technology,Water Science And Technology,0,Interdiscipline,0,Interdiscipline,Black


In [8]:
mapping['pr'] = mapping['jid'].map(vid_pr)

In [9]:
top = mapping.sort_values('pr', ascending=False).groupby('disc_name').head(20)

In [10]:
top = top.sort_values(['disc_id', 'pr'], ascending=[1, 0])[['disc_name', 'jname', 'pr', 'jid']]

In [11]:
len(top)

280

In [50]:
top.to_csv(mag.get_path('top_journals.csv'), index=None)